In [1]:
import timm
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import nn, optim
from torchvision.datasets import FashionMNIST
import torch.nn.functional as F
import os


import wandb

wandb.finish()
# Initialize a new wandb run
wandb.init(project='cifar10_classification', dir="./wandb")

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Currently logged in as: nomad_at_you. Use `wandb login --relogin` to force relogin


## Hyperparameters

In [2]:
config = wandb.config
config.learning_rate = 0.001
config.epochs = 10
config.batch_size = 64
config.model_name = 'resnet18'


## Data Handling

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])



trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


## Random Neural Network Setup

In [4]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
net = CustomCNN()
net = net.to('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

## Training

In [6]:
for epoch in range(15):  # number of epochs
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')


Epoch 1, Loss: 1.4931088567084974
Epoch 2, Loss: 1.0818937967347977
Epoch 3, Loss: 0.9157069305629681
Epoch 4, Loss: 0.8229182821405513
Epoch 5, Loss: 0.7578941752081332
Epoch 6, Loss: 0.7068333972597975
Epoch 7, Loss: 0.6653002266917387
Epoch 8, Loss: 0.6400511639045022
Epoch 9, Loss: 0.6093790232754119
Epoch 10, Loss: 0.5850792161910735
Epoch 11, Loss: 0.5635487074818453
Epoch 12, Loss: 0.5417516734784521
Epoch 13, Loss: 0.5251358732619249
Epoch 14, Loss: 0.50993497343853
Epoch 15, Loss: 0.5033662905702201


## Evaluation

In [7]:
net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total}%')

Accuracy of the network on the 10000 test images: 80%


## RestNet Setup

In [8]:
model = timm.create_model('resnet18', pretrained=True, num_classes=10)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## ResNet Training

In [10]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")
    wandb.log({"epoch": epoch, "loss": running_loss / len(trainloader)})

Epoch 1, Loss: 1.1421839438588417
Epoch 2, Loss: 0.7442038512367117
Epoch 3, Loss: 0.6482111285333438
Epoch 4, Loss: 0.5777236294106144
Epoch 5, Loss: 0.5338637189144064
Epoch 6, Loss: 0.49681411786457463
Epoch 7, Loss: 0.46693367693963866
Epoch 8, Loss: 0.43838154071051144
Epoch 9, Loss: 0.41899434309405137
Epoch 10, Loss: 0.39814573387279534


## RestNet Evaluation

In [11]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    wandb.log({"test_accuracy": 100 * correct // total})
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total}%')


Accuracy of the network on the 10000 test images: 84%


In [12]:
wandb.watch(model, log='all')


[]

In [13]:
torch.save(model.state_dict(), 'model.pth')
wandb.save('model.pth')


['./wandb/wandb/run-20231210_202952-i73uhteb/files/model.pth']